# Linear Regression

In [1]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import polars as pl
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
import matplotlib.pyplot as plt
import plotly.express as px


In [4]:
# Load the skaters dataset
skaters_df = pd.read_excel("../../data/instat/SkatersCleaned.xlsx")

# Display the DataFrame as a scrollable HTML table
display(HTML(skaters_df.to_html(index=False, max_rows=5, max_cols=None)))

Shirt number,Player,Position,Games played,All shifts,Goals,First assist,Second assist,Assists,Points,+/-,Scoring chances,Penalties drawn,Faceoffs,Faceoffs won,"Faceoffs won, %",Hits,Shots,Shots on goal,Blocked shots,Power play shots,Short-handed shots,Passes to the slot,Faceoffs in DZ,Faceoffs won in DZ,"Faceoffs won in DZ, %",Faceoffs in NZ,Faceoffs won in NZ,"Faceoffs won in NZ, %",Faceoffs in OZ,Faceoffs won in OZ,"Faceoffs won in OZ, %",Puck touches,Plus,Minus,Penalties,Faceoffs lost,Hits against,Error leading to goal,Dump ins,Dump outs,Team goals when on ice,Opponent's goals when on ice,Power play,Successful power play,Short-handed,Penalty killing,Missed shots,% shots on goal,Slapshot,Wrist shot,Shootouts,Shootouts scored,Shootouts missed,1-on-1 shots,1-on-1 goals,"Shots conversion 1 on 1, %",Positional attack shots,Shots 5 v 5,Counter-attack shots,xG per shot,xG (Expected goals),xG per goal,Net xG (xG player on - opp. team's xG),Team xG when on ice,Opponent's xG when on ice,xG conversion,CORSI,CORSI-,CORSI+,"CORSI for, %",Fenwick for,Fenwick against,"Fenwick for, %",Puck battles,Puck battles won,"Puck battles won, %",Puck battles in DZ,Puck battles in NZ,Puck battles in OZ,Shots blocking,Dekes,Dekes successful,Dekes unsuccessful,"Dekes successful, %",Passes,Accurate passes,"Accurate passes, %",Pre-shots passes,Pass receptions,Scoring chances - total,Scoring chances - scored,Scoring chances missed,Scoring chances saved,"Scoring Chances, %",Inner slot shots - total,Inner slot shots - scored,Inner slot shots - missed,Inner slot shots - saved,"Inner slot shots, %",Outer slot shots - total,Outer slot shots - scored,Outer slot shots - missed,Outer slot shots - saved,"Outer slot shots, %",Blocked shots from the slot,Blocked shots outside of the slot,Takeaways,Puck retrievals after shots,Opponent’s dump-in retrievals,Takeaways in DZ,Loose puck recovery,Takeaways in NZ,Takeaways in OZ,EV DZ retrievals,Puck losses,Puck losses in DZ,EV OZ retrievals,Puck losses in NZ,Power play retrievals,Penalty kill retrievals,Puck losses in OZ,Entries,Entries via pass,Entries via dump in,Entries via stickhandling,Breakouts,Breakouts via pass,Breakouts via dump out,Breakouts via stickhandling,National team,Height,Weight,Contract,Active hand,Type,Time on ice (Minutes),Time on ice (Seconds),Penalty time (Minutes),Penalty time (Seconds),Puck control time (Minutes),Puck control time (Seconds),Power play time (Minutes),Power play time (Seconds),Short-handed time (Minutes),Short-handed time (Seconds),Playing in attack (Minutes),Playing in attack (Seconds),Playing in defense (Minutes),Playing in defense (Seconds),OZ possession (Minutes),OZ possession (Seconds),NZ possession (Minutes),NZ possession (Seconds),DZ possession (Minutes),DZ possession (Seconds)
6,Alyssa Wilson,F,23,17.0,0.04,0.04,0.04,0.09,0.13,-1.04,0.04,0.04,0.09,0.04,0.50,0.13,2.00,0.74,0.61,0.00,0.0,0.17,0.04,0.04,1.00,0.04,0.00,0.00,0.0,0.00,0.00,57.0,0.22,1.26,0.17,0.04,0.04,0.96,1.61,1.96,0.22,1.26,0.00,0.00,0.00,0.00,0.65,0.37,0.74,0.87,0.0,0.0,0.0,0.00,0.0,0.0,1.17,2.00,0.04,0.31,0.05,0.29,-0.75,0.46,1.22,0.0091,-8.0,18,10.0,0.35,7.0,14.0,0.35,10.0,5.2,0.54,7.0,1.61,1.39,0.78,0.26,0.22,0.04,0.83,12.0,10.0,0.83,0.22,6.0,0.26,0.04,0.04,0.17,0.17,0.04,0.04,0.00,0.00,1.0,0.22,0.0,0.04,0.17,0.0,0.00,0.61,7.0,1.87,1.09,5.20,1.48,1.61,0.39,1.39,4.20,1.30,0.48,1.04,0.00,0.00,1.87,1.48,0.52,0.22,0.74,2.30,1.48,0.13,0.70,0,0,0.0,0,Right,Even Strength,15,20,0,26,0,16,0,0,0,0,6,15,7,7,4,56,3,8,7,16
12,Ella Bynan,F,24,16.0,0.00,0.08,0.13,0.21,0.21,-0.58,0.38,0.13,9.00,4.30,0.48,0.00,1.54,1.17,0.21,0.00,0.0,0.25,3.80,1.79,0.47,2.80,1.38,0.49,2.4,1.17,0.48,55.0,0.21,0.79,0.04,4.70,0.04,0.08,1.79,0.50,0.21,0.79,0.00,0.00,0.00,0.00,0.17,0.76,0.54,0.38,0.0,0.0,0.0,0.00,0.0,0.0,0.50,1.54,0.08,1.33,0.12,0.00,-0.60,0.43,1.03,0.0000,-7.0,17,9.0,0.36,7.0,13.0,0.35,14.0,5.9,0.42,6.0,2.00,5.70,0.92,0.63,0.42,0.21,0.67,9.0,7.0,0.71,0.29,8.0,0.71,0.00,0.08,0.63,0.00,0.38,0.00,0.04,0.33,0.0,0.33,0.0,0.04,0.29,0.

In [5]:
# Drop unneeded columns
skaters_df.drop(['Player', 'Contract'], axis=1, inplace=True)

# Find remaining categorical columns and one‐hot encode them
cat_cols = skaters_df.select_dtypes(include=['object']).columns.tolist()
skaters_df = pd.get_dummies(skaters_df, columns=cat_cols)

display(HTML(skaters_df.to_html(index=False, max_rows=5, max_cols=None)))

Shirt number,Games played,All shifts,Goals,First assist,Second assist,Assists,Points,+/-,Scoring chances,Penalties drawn,Faceoffs,Faceoffs won,"Faceoffs won, %",Hits,Shots,Shots on goal,Blocked shots,Power play shots,Short-handed shots,Passes to the slot,Faceoffs in DZ,Faceoffs won in DZ,"Faceoffs won in DZ, %",Faceoffs in NZ,Faceoffs won in NZ,"Faceoffs won in NZ, %",Faceoffs in OZ,Faceoffs won in OZ,"Faceoffs won in OZ, %",Puck touches,Plus,Minus,Penalties,Faceoffs lost,Hits against,Error leading to goal,Dump ins,Dump outs,Team goals when on ice,Opponent's goals when on ice,Power play,Successful power play,Short-handed,Penalty killing,Missed shots,% shots on goal,Slapshot,Wrist shot,Shootouts,Shootouts scored,Shootouts missed,1-on-1 shots,1-on-1 goals,"Shots conversion 1 on 1, %",Positional attack shots,Shots 5 v 5,Counter-attack shots,xG per shot,xG (Expected goals),xG per goal,Net xG (xG player on - opp. team's xG),Team xG when on ice,Opponent's xG when on ice,xG conversion,CORSI,CORSI-,CORSI+,"CORSI for, %",Fenwick for,Fenwick against,"Fenwick for, %",Puck battles,Puck battles won,"Puck battles won, %",Puck battles in DZ,Puck battles in NZ,Puck battles in OZ,Shots blocking,Dekes,Dekes successful,Dekes unsuccessful,"Dekes successful, %",Passes,Accurate passes,"Accurate passes, %",Pre-shots passes,Pass receptions,Scoring chances - total,Scoring chances - scored,Scoring chances missed,Scoring chances saved,"Scoring Chances, %",Inner slot shots - total,Inner slot shots - scored,Inner slot shots - missed,Inner slot shots - saved,"Inner slot shots, %",Outer slot shots - total,Outer slot shots - scored,Outer slot shots - missed,Outer slot shots - saved,"Outer slot shots, %",Blocked shots from the slot,Blocked shots outside of the slot,Takeaways,Puck retrievals after shots,Opponent’s dump-in retrievals,Takeaways in DZ,Loose puck recovery,Takeaways in NZ,Takeaways in OZ,EV DZ retrievals,Puck losses,Puck losses in DZ,EV OZ retrievals,Puck losses in NZ,Power play retrievals,Penalty kill retrievals,Puck losses in OZ,Entries,Entries via pass,Entries via dump in,Entries via stickhandling,Breakouts,Breakouts via pass,Breakouts via dump out,Breakouts via stickhandling,National team,Height,Weight,Time on ice (Minutes),Time on ice (Seconds),Penalty time (Minutes),Penalty time (Seconds),Puck control time (Minutes),Puck control time (Seconds),Power play time (Minutes),Power play time (Seconds),Short-handed time (Minutes),Short-handed time (Seconds),Playing in attack (Minutes),Playing in attack (Seconds),Playing in defense (Minutes),Playing in defense (Seconds),OZ possession (Minutes),OZ possession (Seconds),NZ possession (Minutes),NZ possession (Seconds),DZ possession (Minutes),DZ possession (Seconds),Position_D,Position_F,Active hand_Left,Active hand_Right,Type_Even Strength,Type_Penalty Kill,Type_Power Play,Type_Total
6,23,17.0,0.04,0.04,0.04,0.09,0.13,-1.04,0.04,0.04,0.09,0.04,0.50,0.13,2.00,0.74,0.61,0.00,0.0,0.17,0.04,0.04,1.00,0.04,0.00,0.00,0.0,0.00,0.00,57.0,0.22,1.26,0.17,0.04,0.04,0.96,1.61,1.96,0.22,1.26,0.00,0.00,0.00,0.00,0.65,0.37,0.74,0.87,0.0,0.0,0.0,0.00,0.0,0.0,1.17,2.00,0.04,0.31,0.05,0.29,-0.75,0.46,1.22,0.0091,-8.0,18,10.0,0.35,7.0,14.0,0.35,10.0,5.2,0.54,7.0,1.61,1.39,0.78,0.26,0.22,0.04,0.83,12.0,10.0,0.83,0.22,6.0,0.26,0.04,0.04,0.17,0.17,0.04,0.04,0.00,0.00,1.0,0.22,0.0,0.04,0.17,0.0,0.00,0.61,7.0,1.87,1.09,5.20,1.48,1.61,0.39,1.39,4.20,1.30,0.48,1.04,0.00,0.00,1.87,1.48,0.52,0.22,0.74,2.30,1.48,0.13,0.70,0,0,0.0,15,20,0,26,0,16,0,0,0,0,6,15,7,7,4,56,3,8,7,16,False,True,False,True,True,False,False,False
12,24,16.0,0.00,0.08,0.13,0.21,0.21,-0.58,0.38,0.13,9.00,4.30,0.48,0.00,1.54,1.17,0.21,0.00,0.0,0.25,3.80,1.79,0.47,2.80,1.38,0.49,2.4,1.17,0.48,55.0,0.21,0.79,0.04,4.70,0.04,0.08,1.79,0.50,0.21,0.79,0.00,0.00,0.00,0.00,0.17,0.76,0.54,0.38,0.0,0.0,0.0,0.00,0.0,0.0,0.50,1.54,0.08,1.33,0.12,0.00,-0.60,0.43,1.03,0.0000,-7.0,17,9.0,0.36,7.0,13.0,0.35,14.0,5.9,0.42,6.0,2.00,5.70,0.92,0.63,0.42,0.21,0.67,9.0,7.0,0.71,0.29,8

In [6]:
# Drop identifier columns
skaters_df.drop(['Shirt number'], axis=1, inplace=True)

# Define the categorical columns that should be encoded (only include those present)
cat_to_encode = ['Position', 'National team', 'Contract', 'Active hand', 'Type']
cat_to_encode = [col for col in cat_to_encode if col in skaters_df.columns]

# One-hot encode the selected categorical columns
skaters_df = pd.get_dummies(skaters_df, columns=cat_to_encode)

# Display a preview
skaters_df.head()

,Games played,All shifts,Goals,First assist,Second assist,Assists,Points,+/-,Scoring chances,Penalties drawn,...,DZ possession (Minutes),DZ possession (Seconds),Position_D,Position_F,Active hand_Left,Active hand_Right,Type_Even Strength,Type_Penalty Kill,Type_Power Play,Type_Total
0,23,17.0,0.04,0.04,0.04,0.09,0.13,-1.04,0.04,0.04,...,7,16,False,True,False,True,True,False,False,False
1,24,16.0,0.00,0.08,0.13,0.21,0.21,-0.58,0.38,0.13,...,6,40,False,True,False,True,True,False,False,False
2,24,12.0,0.08,0.00,0.00,0.00,0.08,-0.29,0.21,0.33,...,5,20,False,True,False,True,True,False,False,False
3,23,7.0,0.04,0.00,0.00,0.00,0.04,-0.13,0.04,0.04,...,2,54,False,True,False,True,True,False,False,False
4,24,15.0,0.13,0.04,0.00,0.04,0.17,-0.21,0.17,0.08,...,5,18,False,True,False,True,True,False,False,False


## Linear Regression based on Goals

In [7]:
# Prepare data for linear regression
# Define target and columns to drop
goal_target = "Goals"
drop_cols = [
    goal_target,
    "Points",
    "Scoring chances",
    "Shots conversion 1 on 1, %",
    "Inner slot shots - scored",
    "Outer slot shots - scored",
    "xG per goal",
    "+/-",
    "Team goals when on ice",
    "Plus"
]

# Features and target
X = skaters_df.drop(columns=drop_cols, errors="ignore")
y = skaters_df[goal_target]

# Set up cross‐validation and model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()

# Evaluate with MSE
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring="neg_mean_squared_error")
print(f"CV mean MSE: {mse_scores.mean():.4f}")

# Train final model on all data
model.fit(X, y)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

CV mean MSE: 0.0027
Coefficients: {'Games played': 7.069580470361684e-05, 'All shifts': 0.005867295574171709, 'First assist': -0.01588138776865823, 'Second assist': -0.015157294941819083, 'Assists': -0.030867748698149414, 'Penalties drawn': -0.03958873811525597, 'Faceoffs': -0.0361909209624713, 'Faceoffs won': 0.03402733051074924, 'Faceoffs won, %': -0.03040309293654694, 'Hits': -0.02464933104013614, 'Shots': 0.010550270699011929, 'Shots on goal': 0.004901329496251517, 'Blocked shots': -0.007633388004332612, 'Power play shots': 0.014631897244066154, 'Short-handed shots': -0.024507128135442285, 'Passes to the slot': -0.03456955807779314, 'Faceoffs in DZ': 0.021855460525521973, 'Faceoffs won in DZ': 0.0287112565592068, 'Faceoffs won in DZ, %': 0.013359985855443776, 'Faceoffs in NZ': 0.009973171999984622, 'Faceoffs won in NZ': 0.0017814504365000882, 'Faceoffs won in NZ, %': -0.004404747604558352, 'Faceoffs in OZ': -0.003925021096413668, 'Faceoffs won in OZ': -0.00015235906386890097, 'Face

In [8]:
# Create a DataFrame to display coefficients
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})

# Sort the DataFrame by absolute coefficient value
coef_df['Absolute Coefficient'] = abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values(by='Absolute Coefficient', ascending=False)

# Get the top N coefficients
top_coef_df = coef_df.head(10).copy()

print(top_coef_df)

                          Feature  Coefficient  Absolute Coefficient
47        Positional attack shots    -0.081545              0.081545
100               Takeaways in OZ     0.069074              0.069074
80       Scoring chances - scored     0.061708              0.061708
126     Power play time (Minutes)    -0.061534              0.061534
105             Puck losses in NZ    -0.055529              0.055529
41                     Wrist shot     0.053970              0.053970
109                       Entries     0.050843              0.050843
30          Error leading to goal    -0.049997              0.049997
33   Opponent's goals when on ice     0.049649              0.049649
84       Inner slot shots - total     0.045436              0.045436


In [9]:
# Get coefficients and their corresponding column names
coefficients = model.coef_
column_names = X.columns

# Create a DataFrame to display coefficients
coef_df = pd.DataFrame({'Feature': column_names, 'Coefficient': coefficients})

# Sort the DataFrame by absolute coefficient value
coef_df['Absolute Coefficient'] = abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values(by='Absolute Coefficient', ascending=False)

# Print the top N coefficients
N = 10  # Number of top coefficients to display
print(f"Top {N} coefficients:")
print(coef_df[['Feature', 'Coefficient']].head(N))

Top 10 coefficients:
                          Feature  Coefficient
47        Positional attack shots    -0.081545
100               Takeaways in OZ     0.069074
80       Scoring chances - scored     0.061708
126     Power play time (Minutes)    -0.061534
105             Puck losses in NZ    -0.055529
41                     Wrist shot     0.053970
109                       Entries     0.050843
30          Error leading to goal    -0.049997
33   Opponent's goals when on ice     0.049649
84       Inner slot shots - total     0.045436


In [10]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import plotly.express as px
import pandas as pd

# Select the top N features based on absolute coefficient values
N = 10  # You can adjust the number of top features to use
top_features = coef_df.head(N)['Feature'].tolist()

# Prepare the data using only the top features
X_top = X[top_features]

# Add a constant to the independent variables
X_top = sm.add_constant(X_top)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train the multiple linear regression model
model = sm.OLS(y_train, X_train).fit()

# Print the model summary
print(model.summary())

# Make predictions on the test set
y_pred = model.predict(X_test)

# Create a DataFrame for actual vs predicted values
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Plotting actual vs predicted values using Plotly
fig = px.scatter(
    predictions_df,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Goals Per Game',
    labels={'Actual': 'Actual', 'Predicted': 'Predicted'}
)

# Add a diagonal line representing perfect predictions
fig.add_trace(
    px.line(
        x=[predictions_df['Actual'].min(), predictions_df['Actual'].max()],
        y=[predictions_df['Actual'].min(), predictions_df['Actual'].max()]
    ).data[0]
)

fig.show()

                            OLS Regression Results                            
Dep. Variable:                  Goals   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     546.9
Date:                Wed, 07 May 2025   Prob (F-statistic):           9.55e-50
Time:                        14:59:27   Log-Likelihood:                 206.62
No. Observations:                  64   AIC:                            -391.2
Df Residuals:                      53   BIC:                            -367.5
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [13]:
# Create a Coefficient Magnitude Plot
top_coef_df['Magnitude'] = top_coef_df['Coefficient'].abs()

fig = px.scatter(
    top_coef_df,
    x='Coefficient',
    y='Feature',
    size='Magnitude',
    color='Coefficient',
    hover_data=['Coefficient', 'Magnitude'],
    title='Coefficient Magnitude Plot for Goals',
    labels={'Coefficient': 'Coefficient Value', 'Feature': 'Feature'},
    color_continuous_scale=px.colors.diverging.RdBu
)

fig.update_layout(
    xaxis_title="Coefficient Value",
    yaxis_title="Feature",
    title_x=0.5,
    height=600,
    width=800
)

fig.show()

/var/folders/fn/1h19dxrs6m105yhjv2k86ryr0000gn/T/ipykernel_2748/1590220741.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
# Create a bar graph of the top coefficients
top_coef_df = coef_df.head(N)

fig = px.bar(
    top_coef_df,
    x='Feature',
    y='Coefficient',
    title=f'Top {N} Variables Influencing Goals',
    labels={'Coefficient': 'Coefficient Value', 'Feature': 'Feature'}
)

fig.show()

## Linear Regression based on Points

In [19]:
# Prepare data for linear regression
# Define target and columns to drop
point_target = "Points"
drop_cols = [
    point_target,
    "Goals",
    "Assists",
    "First assist",
    "Second assist",
    "Scoring chances - scored",
    "Inner slot shots - scored",
    "Outer slot shots - scored",
    "Shots conversion 1 on 1, %",
    "xG per goal",
    "Passes to the slot",
    "Pre-shots passes",
    "Accurate passes",
    "Accurate passes, %",
    "+/-",
    "Team goals when on ice",
    "Plus"
]

# Features and target
X = skaters_df.drop(columns=drop_cols, errors="ignore")
y = skaters_df[point_target]

# Set up cross‐validation and model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()

# Evaluate with MSE
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring="neg_mean_squared_error")
print(f"CV mean MSE: {mse_scores.mean():.4f}")

# Train final model on all data
model.fit(X, y)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

CV mean MSE: 0.0059
Coefficients: {'Shirt number': -0.00012640324571202405, 'Games played': 0.0001773379743115046, 'All shifts': 0.011285246962939866, 'Scoring chances': 0.04400840324457036, 'Penalties drawn': -0.08003262177780558, 'Faceoffs': 0.0010510800429999658, 'Faceoffs won': -0.01850288785048332, 'Faceoffs won, %': 0.01205557064270658, 'Hits': -0.01982885122598564, 'Shots': 0.014188287756333512, 'Shots on goal': 0.02118022556000999, 'Blocked shots': -0.05447805209072096, 'Power play shots': -0.005494501114913758, 'Short-handed shots': -0.0005416789711540533, 'Faceoffs in DZ': 0.025891187971613987, 'Faceoffs won in DZ': -0.00967873993292426, 'Faceoffs won in DZ, %': 0.015809076271973782, 'Faceoffs in NZ': -0.028745748150210398, 'Faceoffs won in NZ': 0.006838111960141955, 'Faceoffs won in NZ, %': -0.02615792912914041, 'Faceoffs in OZ': -0.0041008332676301905, 'Faceoffs won in OZ': -0.0031104355878269607, 'Faceoffs won in OZ, %': 0.008604076759542511, 'Puck touches': 0.009140996078

In [14]:
# Get coefficients and their corresponding column names
coefficients = model.coef_
column_names = X.columns

# Create a DataFrame to display coefficients
points_coef_df = pd.DataFrame({'Feature': column_names, 'Coefficient': coefficients})

# Sort the DataFrame by absolute coefficient value
points_coef_df['Absolute Coefficient'] = abs(coef_df['Coefficient'])
points_coef_df = coef_df.sort_values(by='Absolute Coefficient', ascending=False)

# Print the top N coefficients
N = 10  # Number of top coefficients to display
print(f"Top {N} coefficients:")
print(points_coef_df[['Feature', 'Coefficient']].head(N))

Top 10 coefficients:
                          Feature  Coefficient
48        Positional attack shots    -0.083702
101               Takeaways in OZ     0.072780
127     Power play time (Minutes)    -0.065636
81       Scoring chances - scored     0.060813
42                     Wrist shot     0.050856
80        Scoring chances - total     0.049697
106             Puck losses in NZ    -0.048496
34   Opponent's goals when on ice     0.048391
114                     Breakouts     0.047157
85       Inner slot shots - total     0.045439


In [15]:

# Create a DataFrame for actual vs predicted values
points_predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Plotting actual vs predicted values using Plotly
fig = px.scatter(
    points_predictions_df,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Points Per Game',
    labels={'Actual': 'Actual Points', 'Predicted': 'Predicted Points'}
)

# Add a diagonal line representing perfect predictions
fig.add_trace(
    px.line(
        x=[points_predictions_df['Actual'].min(), points_predictions_df['Actual'].max()],
        y=[points_predictions_df['Actual'].min(), points_predictions_df['Actual'].max()]
    ).data[0]
)

fig.show()

## Linear Regression of Assists

In [16]:
# Prepare data for linear regression
# Define target and columns to drop
assists_target = "Assists"
drop_cols = [
    assists_target,
    "First assists",
    "Second assists",
    "Points",
    "Passes",
    "Accurate passes",
    "+/-",
    "Team goals when on ice",
    "Plus",
    "Accurate passes, %",
    "Shirt number",
    "Player"
    "OZ possession (Seconds)",
    "NZ possession (Seconds)",
    "DZ possession (Seconds)",
    "Short-handed time (Seconds)"
]

# Features and target
X = skaters_df.drop(columns=drop_cols, errors="ignore")
y = skaters_df[assists_target]

# Set up cross‐validation and model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()

# Evaluate with MSE
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring="neg_mean_squared_error")
print(f"CV mean MSE: {mse_scores.mean():.4f}")

# Train final model on all data
model.fit(X, y)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

CV mean MSE: 0.0022
Coefficients: {'Games played': -0.00025281700897590104, 'All shifts': -0.0012734204298302156, 'Goals': -0.03444002596613438, 'First assist': 0.043361111980582144, 'Second assist': 0.04348859447334944, 'Scoring chances': -0.007898287807405056, 'Penalties drawn': -0.031034114723228534, 'Faceoffs': 0.041998361705233564, 'Faceoffs won': -0.0468864009595131, 'Faceoffs won, %': 0.022689915408114273, 'Hits': 0.007758025254839252, 'Shots': 0.01766824455202605, 'Shots on goal': 0.02456513790871337, 'Blocked shots': -0.05610469101932829, 'Power play shots': -0.00789694398563821, 'Short-handed shots': 0.03767697713333591, 'Passes to the slot': 0.004872864145091551, 'Faceoffs in DZ': -0.021283661158730854, 'Faceoffs won in DZ': -0.029327650683256892, 'Faceoffs won in DZ, %': -0.0001261121243340789, 'Faceoffs in NZ': -0.026516823902031833, 'Faceoffs won in NZ': 0.0006847182264845542, 'Faceoffs won in NZ, %': -0.0019754262777716136, 'Faceoffs in OZ': 0.0016344877683284083, 'Faceo

In [17]:


# Create a DataFrame for actual vs predicted values
assists_predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Plotting actual vs predicted values using Plotly
fig = px.scatter(
    assists_predictions_df,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Assists Per Game',
    labels={'Actual': 'Actual Assists', 'Predicted': 'Predicted Assists'}
)

# Add a diagonal line representing perfect predictions
fig.add_trace(
    px.line(
        x=[assists_predictions_df['Actual'].min(), assists_predictions_df['Actual'].max()],
        y=[assists_predictions_df['Actual'].min(), assists_predictions_df['Actual'].max()]
    ).data[0]
)

fig.show()

## Linear Regression on Scoring chances

In [55]:
skaters_df.columns.tolist()

['Shirt number',
 'Games played',
 'All shifts',
 'Goals',
 'First assist',
 'Second assist',
 'Assists',
 'Points',
 '+/-',
 'Scoring chances',
 'Penalties drawn',
 'Faceoffs',
 'Faceoffs won',
 'Faceoffs won, %',
 'Hits',
 'Shots',
 'Shots on goal',
 'Blocked shots',
 'Power play shots',
 'Short-handed shots',
 'Passes to the slot',
 'Faceoffs in DZ',
 'Faceoffs won in DZ',
 'Faceoffs won in DZ, %',
 'Faceoffs in NZ',
 'Faceoffs won in NZ',
 'Faceoffs won in NZ, %',
 'Faceoffs in OZ',
 'Faceoffs won in OZ',
 'Faceoffs won in OZ, %',
 'Puck touches',
 'Plus',
 'Minus',
 'Penalties',
 'Faceoffs lost',
 'Hits against',
 'Error leading to goal',
 'Dump ins',
 'Dump outs',
 'Team goals when on ice',
 "Opponent's goals when on ice",
 'Power play',
 'Successful power play',
 'Short-handed',
 'Penalty killing',
 'Missed shots',
 '% shots on goal',
 'Slapshot',
 'Wrist shot',
 'Shootouts',
 'Shootouts scored',
 'Shootouts missed',
 '1-on-1 shots',
 '1-on-1 goals',
 'Shots conversion 1 on 1, %

In [56]:
# Prepare data for linear regression
# Define target and columns to drop
target = "Scoring chances"
drop_cols = [
    target,
    "Shirt number",
    "Player",
    "Position",
    "Nationality",
    "Time on ice (Seconds)",
    "Time on ice per game (Seconds)",
    "Penalty time (Seconds)",
    "Puck control time (Seconds)",
    "Power play time (Seconds)",
    "Short-handed time (Seconds)",
    "Playing in attack (Seconds)",
    "Playing in defense (Seconds)",
    "OZ possession (Seconds)",
    "NZ possession (Seconds)",
    "DZ possession (Seconds)",
    "Scoring chances missed",
    "Scoring chances saved",
    "Scoring chances - total",
    "Inner slot shots - total",
    "Outer slot shots - total"
]

# Features and target
X = skaters_df.drop(columns=drop_cols, errors="ignore")
y = skaters_df[target]

# Set up cross‐validation and model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()

# Evaluate with MSE
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring="neg_mean_squared_error")
print(f"CV mean MSE: {mse_scores.mean():.4f}")

# Train final model on all data
model.fit(X, y)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

CV mean MSE: 0.0061
Coefficients: {'Games played': 0.0005391087841089893, 'All shifts': 0.0010478952494591513, 'Goals': 0.060216643235750426, 'First assist': -0.005548611390783817, 'Second assist': -0.02104721379676543, 'Assists': -0.02447978740918578, 'Points': 0.03680474904215518, '+/-': -0.0027199353254385482, 'Penalties drawn': 0.0005172545597186655, 'Faceoffs': 0.03302136399983441, 'Faceoffs won': -0.00593678787682474, 'Faceoffs won, %': -0.014675858392432115, 'Hits': -0.023372088163285874, 'Shots': 0.014338225921819697, 'Shots on goal': 0.050413256810525664, 'Blocked shots': -0.04785692917357224, 'Power play shots': 0.03966305602029688, 'Short-handed shots': 0.002935372005187991, 'Passes to the slot': -0.05659218017564237, 'Faceoffs in DZ': -0.0675027639135742, 'Faceoffs won in DZ': 0.030792791548342666, 'Faceoffs won in DZ, %': 0.027840772856245474, 'Faceoffs in NZ': -0.03421859378485903, 'Faceoffs won in NZ': 0.005699737723694439, 'Faceoffs won in NZ, %': -0.008349428843112235,

In [57]:
# Create a DataFrame for actual vs predicted values
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Plotting actual vs predicted values using Plotly
fig = px.scatter(
    predictions_df,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Scoring Chances Per Game',
    labels={'Actual': 'Actual Scoring Chances', 'Predicted': 'Predicted Scoring Chances'}
)

# Add a diagonal line representing perfect predictions
fig.add_trace(
    px.line(
        x=[predictions_df['Actual'].min(), predictions_df['Actual'].max()],
        y=[predictions_df['Actual'].min(), predictions_df['Actual'].max()]
    ).data[0]
)

fig.show()